In [1]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

In [2]:
import tensorflow as tf

import keras.models 

from keras.models import Model
from keras.layers import Dense

import keras_metrics

Using TensorFlow backend.


In [3]:
train_data = pd.read_csv("../Archive/people_train.csv")
train_data.head()

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3
0,0,0,0.143663,-0.351520,0.142857,1.846564,0,-0.421181,0.108757,1.005063,...,1.039112,-0.432306,-0.312382,-0.721217,0.260517,0.183046,0,0,1,0
1,2,0,-0.113321,-0.450280,0.666667,1.736781,0,-0.212764,-0.544194,0.003134,...,-0.615425,-0.352255,-0.489128,-0.174106,-0.888673,-0.330729,0,0,1,0
2,3,0,-0.106171,1.475538,0.308642,1.736781,0,-0.844768,-0.255389,0.003134,...,0.024637,1.488931,1.455079,1.819491,1.693087,1.667765,0,0,1,0
3,4,0,-0.044296,-0.499660,1.000000,-0.349103,0,1.884350,0.410118,2.006993,...,-0.615425,-0.432306,-0.524477,-0.721217,-0.888673,-0.996894,0,1,0,0
4,5,0,-0.117145,-0.450280,0.333333,-0.513778,0,-0.046556,-0.318172,0.003134,...,0.399049,-0.432306,-0.453778,-0.721217,-0.444106,-0.330729,0,1,0,0


In [4]:
val_data = pd.read_csv("../Archive/people_validate.csv")
val_data.head()

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3
0,17,0,0.258922,-0.499660,1.000000,1.078080,0,0.757848,0.749150,1.005063,...,-0.615425,-0.432306,-0.524477,-0.721217,-0.888673,-0.996894,0,0,1,0
1,21,0,-0.116272,-0.005861,0.190476,1.462322,0,-0.740466,-0.293059,-0.998796,...,0.024637,-0.192152,0.076460,0.373006,0.928479,0.849211,0,0,1,0
2,25,0,-0.059747,5.154346,0.282609,-0.678453,0,-0.865404,-0.142378,0.003134,...,3.101643,4.690994,5.308143,2.573691,2.386152,2.300590,0,1,0,0
3,28,0,-0.115467,-0.474970,0.000000,1.352539,0,1.525107,1.778803,-0.998796,...,-0.615425,-0.512358,-0.453778,-0.721217,-0.444106,-0.576591,0,0,1,0
4,34,0,-0.087982,-0.474970,0.000000,0.364489,0,0.384911,0.259438,-0.998796,...,-0.615425,-0.512358,-0.453778,-0.721217,-0.444106,-0.576591,0,0,1,0


In [5]:
original_test = pd.read_csv("../test.csv")

In [6]:
test_data = pd.read_csv("../people_test_v2.csv").reset_index()
original_test = pd.read_csv("../test.csv")
test_data["customer"] = original_test[original_test["category"] == 0].reset_index()["customer"]
test_data.head()

,index,Unnamed: 0,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3,customer
0,0,0,0.321480,-0.49966,0.00,1.023189,0,1.113151,-0.104708,0.003134,...,-0.512358,-0.489128,-0.721217,-0.888673,-0.996894,0,0,1,0,91000000
1,1,2,-0.109433,-0.42559,0.25,-0.239320,0,-0.302809,-0.669761,0.003134,...,-0.432306,-0.418429,-0.721217,-0.184051,-0.156288,0,1,0,0,91000002
2,2,3,-0.091348,-0.42559,0.50,-1.611612,0,-0.429435,-0.230275,0.003134,...,-0.352255,-0.453778,-0.174106,-0.444106,-0.156288,1,0,0,0,91000003
3,3,4,-0.113707,-0.49966,0.00,0.035139,0,0.014225,-0.845555,0.003134,...,-0.512358,-0.489128,-0.721217,-0.888673,-0.996894,0,1,0,0,91000004
4,4,5,-0.115794,-0.40090,0.00,1.956347,0,-0.354210,-0.029367,3.008923,...,-0.512358,-0.347731,-0.721217,0.143580,-0.020980,0,0,1,0,91000005


In [7]:
compare = pd.DataFrame([train_data.std(), val_data.std(), test_data.std()])

In [8]:
pd.options.display.max_columns = 999
compare

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,18,32,33,45,47,50,66,94,113,117,123,134,141,172,176,191,turnover_log,atm_withdrawal_log,atm_deposit_log,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3,index,customer
0,288681.050001,0.099859,1.000000,1.000000,0.307884,1.000000,0.099390,1.000000,1.000000,1.000000,1.000000,0.142317,1.000000,1.000000,0.026914,0.370809,0.417686,0.222006,0.001768,0.295270,0.004145,0.361909,0.014084,0.00125,0.083569,0.139150,0.048223,0.420582,0.006373,0.211752,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.224022,0.499909,0.498061,0.024035,NaN,NaN
1,288619.815134,0.101829,0.575993,1.004418,0.307519,0.999244,0.097780,0.994587,0.992162,0.998627,0.985580,0.141915,0.750952,0.567099,0.025361,0.371109,0.416276,0.223261,0.002500,0.295307,0.003535,0.363468,0.012498,0.00000,0.085910,0.138693,0.048926,0.420186,0.006123,0.211577,1.001257,0.998229,1.001147,0.999812,1.005484,1.003950,1.000436,0.999478,0.999789,0.225092,0.499895,0.498085,0.022492,NaN,NaN
2,288513.643184,NaN,5.876915,1.000901,0.308234,0.999970,0.099898,0.999897,1.000146,1.001294,0.990124,0.142235,3.607522,1.685531,0.027098,0.371033,0.417532,0.222450,0.001118,0.295565,0.004472,0.362092,0.014183,0.00000,0.083775,0.137927,0.049039,0.420492,0.007582,0.211127,1.004862,1.000649,1.000451,0.997412,1.000287,1.001163,0.999344,0.999539,0.999641,0.224732,0.499900,0.498069,0.023336,230996.543663,288513.643184


In [9]:
train_features = train_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
train_labels = train_data["suspicious"].values

In [10]:
val_features = val_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
val_labels = val_data["suspicious"].values

In [11]:
train_features.shape

(640110, 41)

In [12]:
train_features[train_labels == 1].shape

(6448, 41)

In [29]:
augmented_train_features = train_features
augmented_train_labels = train_labels
for i in range(50):
    augmented_train_features = np.concatenate((augmented_train_features, train_features[train_labels == 1]))
    augmented_train_labels = np.concatenate((augmented_train_labels, np.array([1] * train_features[train_labels == 1].shape[0])))

In [30]:
augmented_train_labels.sum()/len(augmented_train_labels)

0.34165671006015524

In [31]:
encoding_dim = train_features.shape[-1]
model_input = keras.models.Input(shape=(encoding_dim,))
x = Dense(32, activation='tanh')(model_input)
x = keras.layers.Dropout(0.1)(x)
x = Dense(16, activation='tanh')(x)
x = keras.layers.Dropout(0.1)(x)
#x = Dense(8, activation='tanh')(x)
#x = keras.layers.Dropout(0.1)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(model_input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [32]:
def cost(preds, labels, tresh):
    investigation_cost = sum(preds > tresh) * 1000
    fine_cost = sum(labels[preds < tresh]) * 60000
    return investigation_cost + fine_cost

In [33]:
for i in range(10):
    history = model.fit(augmented_train_features, tf.keras.utils.to_categorical(augmented_train_labels, 2), epochs=1, batch_size=256, shuffle=True, validation_data=(val_features,  tf.keras.utils.to_categorical(val_labels, 2)),
                verbose=1, )
    
    min_cost = -1
    min_cost_tresh = 0
    preds = model.predict(val_features)[:,1]
    for i in np.linspace(0, 1, 21):
        c = cost(preds, val_labels, i)
        if min_cost == -1:
            min_cost = c
            
        if c <= min_cost:
            min_cost = c
            min_cost_tresh = i
        
    print("COST: " + str(min_cost) + " for tresh. " + str(min_cost_tresh))
    

Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 9s 10us/step - loss: 0.4317 - precision: 0.7472 - recall: 0.6205 - val_loss: 0.2746 - val_precision: 0.0672 - val_recall: 0.6607
COST: 50450000 for tresh. 0.45
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 9s 10us/step - loss: 0.4101 - precision: 0.7624 - recall: 0.6517 - val_loss: 0.3074 - val_precision: 0.0574 - val_recall: 0.7036
COST: 50005000 for tresh. 0.45
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 9s 10us/step - loss: 0.4068 - precision: 0.7636 - recall: 0.6577 - val_loss: 0.2698 - val_precision: 0.0653 - val_recall: 0.6738
COST: 50119000 for tresh. 0.5
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 8s 8us/step - loss: 0.4055 - precision: 0.7641 - recall: 0.6607 - val_loss: 0

In [17]:
full_train_data = pd.concat((train_data, val_data), axis=0)
len(full_train_data)

800146

In [18]:
full_train_data.head()

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,18,32,33,45,47,50,66,94,113,117,123,134,141,172,176,191,turnover_log,atm_withdrawal_log,atm_deposit_log,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3
0,0,0,0.143663,-0.351520,0.142857,1.846564,0,-0.421181,0.108757,1.005063,0.093351,0.128703,0.181939,-0.014727,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1.018328,1.302920,0.797793,1.039112,-0.432306,-0.312382,-0.721217,0.260517,0.183046,0,0,1,0
1,2,0,-0.113321,-0.450280,0.666667,1.736781,0,-0.212764,-0.544194,0.003134,-0.323933,0.138152,-0.070059,-0.036073,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.879020,0.086423,-0.567804,-0.615425,-0.352255,-0.489128,-0.174106,-0.888673,-0.330729,0,0,1,0
2,3,0,-0.106171,1.475538,0.308642,1.736781,0,-0.844768,-0.255389,0.003134,-0.240476,0.333904,-0.068213,-0.035193,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-0.409966,0.254216,-0.187380,0.024637,1.488931,1.455079,1.819491,1.693087,1.667765,0,0,1,0
3,4,0,-0.044296,-0.499660,1.000000,-0.349103,0,1.884350,0.410118,2.006993,-0.323933,0.341578,-0.065363,-0.036527,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.442230,0.394524,-1.573614,-0.615425,-0.432306,-0.524477,-0.721217,-0.888673,-0.996894,0,1,0,0
4,5,0,-0.117145,-0.450280,0.333333,-0.513778,0,-0.046556,-0.318172,0.003134,-0.157020,0.328130,-0.071930,-0.036513,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-2.540978,-0.828882,-1.573614,0.399049,-0.432306,-0.453778,-0.721217,-0.444106,-0.330729,0,1,0,0


In [19]:
full_train_features = full_train_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
full_train_labels = full_train_data["suspicious"].values

In [20]:
full_augmented_train_features = full_train_features
full_augmented_train_labels = full_train_labels
for i in range(50):
    full_augmented_train_features = np.concatenate((full_augmented_train_features, full_train_features[full_train_labels == 1]))
    full_augmented_train_labels = np.concatenate((full_augmented_train_labels, np.array([1] * full_train_features[full_train_labels == 1].shape[0])))

In [21]:
full_augmented_train_labels.sum()/len(full_augmented_train_labels)

0.34348174231346923

In [22]:
encoding_dim = train_features.shape[-1]
model_input = keras.models.Input(shape=(encoding_dim,))
x = Dense(32, activation='tanh')(model_input)
x = keras.layers.Dropout(0.1)(x)
x = Dense(16, activation='tanh')(x)
x = keras.layers.Dropout(0.1)(x)
#x = Dense(8, activation='tanh')(x)
#x = keras.layers.Dropout(0.1)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(model_input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [23]:
for i in range(10):
    history = model.fit(full_augmented_train_features, tf.keras.utils.to_categorical(full_augmented_train_labels, 2), epochs=1, batch_size=256, shuffle=True,
                verbose=1, )   

Epoch 1/1
1206396/1206396 [==============================] - 17s 14us/step - loss: 0.4305 - precision: 0.7498 - recall: 0.6274
Epoch 1/1
1206396/1206396 [==============================] - 15s 13us/step - loss: 0.4110 - precision: 0.7631 - recall: 0.6527
Epoch 1/1
1206396/1206396 [==============================] - 17s 14us/step - loss: 0.4076 - precision: 0.7641 - recall: 0.6574
Epoch 1/1
1206396/1206396 [==============================] - 16s 13us/step - loss: 0.4057 - precision: 0.7651 - recall: 0.6611
Epoch 1/1
1206396/1206396 [==============================] - 16s 13us/step - loss: 0.4041 - precision: 0.7660 - recall: 0.6641
Epoch 1/1
1206396/1206396 [==============================] - 16s 13us/step - loss: 0.4029 - precision: 0.7655 - recall: 0.6652
Epoch 1/1
1206396/1206396 [==============================] - 17s 14us/step - loss: 0.4012 - precision: 0.7664 - recall: 0.6674
Epoch 1/1
1206396/1206396 [==============================] - 17s 14us/step - loss: 0.4003 - precision: 0.7670 -

In [24]:
test_data["predictions"] = model.predict(test_data.drop(["Unnamed: 0","customer","index"], axis=1).values)[:,1]

In [25]:
len(test_data[test_data["predictions"] >= 0.5])

86429

In [26]:
#test_data[test_data["predictions"] >= 0.5]

In [27]:
len(test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]])

86429

In [28]:
test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]].to_csv("../cust_nr_nn.csv", index=False)

In [36]:
test_data[["customer","predictions"]].to_csv("../test_predictions_people_prob_nn.csv", index=False)

In [48]:
val_data = pd.read_csv("../Archive/people_validate.csv")
len(val_data)

160036

In [49]:
val_data["predictions"] = model.predict(val_features)[:,1]

In [50]:
sum(list(map(lambda x: 0 if x < 0.5 else 1, val_data["predictions"])))

16017

In [51]:
sum(val_data["predictions"]) / len(val_data)

0.18865833742845944

In [53]:
val_data[["predictions"]].to_csv("../val_predictions_people_prob_nn.csv", index=False)